In [1]:
import keras_tuner
from tensorflow import keras
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import pathlib
from keras.applications.densenet import DenseNet121
from keras.layers import BatchNormalization, Activation, Dropout, MaxPooling2D,Conv2D
import pandas as pd

2023-04-18 17:51:06.497736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 17:51:07.953382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
size = 96

In [3]:
#EmotioNet
EmotioNet_FACS_X_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_X_train")
EmotioNet_FACS_y_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_Y_train")
EmotioNet_FACS_X_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_X_val")
EmotioNet_FACS_y_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_Y_val")
EmotioNet_FACS_X_test = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_X_test")
EmotioNet_FACS_y_test = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_Y_test")
X_train = EmotioNet_FACS_X_train
X_val = EmotioNet_FACS_X_val
y_train = EmotioNet_FACS_y_train
y_val = EmotioNet_FACS_y_val
X_test = EmotioNet_FACS_X_test
y_test = EmotioNet_FACS_y_test

In [4]:
y_train = EmotioNet_FACS_y_train
y_val = EmotioNet_FACS_y_val

In [5]:
from skimage.transform import resize
def resize_images(images):
    resized_images = []
    for image in images:
        resized_images.append(resize(image, (size, size)))
    return np.asarray(resized_images).astype('float32')

In [6]:
X_train = resize_images(X_train)
X_val = resize_images(X_val)
X_test = resize_images(X_test)

In [7]:
X_train = np.resize(X_train,(len(X_train),size,size))
X_val = np.resize(X_val,(len(X_val),size,size))
X_test = np.resize(X_test,(len(X_test),size,size))

In [8]:
X_train = np.resize(X_train,(len(X_train),size,size,1))
X_val = np.resize(X_val,(len(X_val),size,size,1))
X_test = np.resize(X_test,(len(X_test),size,size,1))

In [9]:
print(np.shape(X_train))
print(np.shape(X_val))
print(np.shape(X_test))

(12629, 96, 96, 1)
(1578, 96, 96, 1)
(1587, 96, 96, 1)


In [10]:
y_train

,1,2,4,5,6,9,10,12,15,17,18,20,24,25,26,28
0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15886,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
15888,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
15890,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
15893,1,1,0,1,0,0,0,0,0,0,1,0,0,1,1,0


In [11]:
np.shape(y_val)

(1578, 16)

In [12]:
np.shape(y_train)

(12629, 16)

In [13]:
NUM_EPOCHS = 20

In [14]:
def change_input_layer_dense(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (4, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False #You can make this trainable if you want.
    updated_model.summary()
    return updated_model

In [15]:
densenet_model = DenseNet121(include_top=False, weights='imagenet')
densenet_updated = change_input_layer_dense(densenet_model, 2, size, size, 1)

2023-04-18 17:51:37.500149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 17:51:37.705543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 17:51:37.706322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

['input_1', 'zero_padding2d', 'conv1/conv', 'conv1/bn', 'conv1/relu', 'zero_padding2d_1', 'pool1', 'conv2_block1_0_bn', 'conv2_block1_0_relu', 'conv2_block1_1_conv', 'conv2_block1_1_bn', 'conv2_block1_1_relu', 'conv2_block1_2_conv', 'conv2_block1_concat', 'conv2_block2_0_bn', 'conv2_block2_0_relu', 'conv2_block2_1_conv', 'conv2_block2_1_bn', 'conv2_block2_1_relu', 'conv2_block2_2_conv', 'conv2_block2_concat', 'conv2_block3_0_bn', 'conv2_block3_0_relu', 'conv2_block3_1_conv', 'conv2_block3_1_bn', 'conv2_block3_1_relu', 'conv2_block3_2_conv', 'conv2_block3_concat', 'conv2_block4_0_bn', 'conv2_block4_0_relu', 'conv2_block4_1_conv', 'conv2_block4_1_bn', 'conv2_block4_1_relu', 'conv2_block4_2_conv', 'conv2_block4_concat', 'conv2_block5_0_bn', 'conv2_block5_0_relu', 'conv2_block5_1_conv', 'conv2_block5_1_bn', 'conv2_block5_1_relu', 'conv2_block5_2_conv', 'conv2_block5_concat', 'conv2_block6_0_bn', 'conv2_block6_0_relu', 'conv2_block6_1_conv', 'conv2_block6_1_bn', 'conv2_block6_1_relu', 'conv

In [16]:
num_classes = 16
NUM_EPOCHS = 20

In [17]:
precision = keras.metrics.Precision(name="Precision")
recall = keras.metrics.Recall(name="Recall")
regularizer = tf.keras.regularizers.l1_l2(0, 0.0001)

In [18]:
from keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

auc_metric = tf.keras.metrics.AUC(
num_thresholds=200, curve="ROC",
summation_method="interpolation",
thresholds=None, multi_label=False
)


In [60]:
num_epoch_scheduler = 10
def scheduler(epoch, lr):
    if epoch < num_epoch_scheduler:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [61]:
import datetime
folder_name = 'logs/DenseNet/Fine-Tuning/Emotion/'
log_folder = folder_name + str(num_epoch_scheduler) + "tf.math.exp(-0.1)"


In [62]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Fine-Tuning/DenseNet',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
callbacks=[scheduler, tensorboard_callback]

In [63]:
model = Sequential()
model.add(densenet_updated)
model.add(Flatten())
model.add(Dense(672, kernel_regularizer=regularizer))
model.add(BatchNormalization(scale=False,
                                center=False))
model.add(Activation('relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002024104233925092),
            loss='binary_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])

In [65]:
model.save("./chkpts/DenseNet_Fine_tuned")

: 

: 

In [64]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=NUM_EPOCHS,
callbacks=callbacks,batch_size=4)

Epoch 1/20
3158/3158 [==============================] - 302s 75ms/step - loss: 0.5165 - Precision: 0.6245 - Recall: 0.0646 - custom_f1: 0.1119 - auc: 0.8068 - accuracy: 0.2327 - val_loss: 1.6331 - val_Precision: 0.5361 - val_Recall: 0.0470 - val_custom_f1: 0.0852 - val_auc: 0.7993 - val_accuracy: 0.3016 - lr: 0.0020
Epoch 2/20
3158/3158 [==============================] - 236s 75ms/step - loss: 0.4550 - Precision: 0.7077 - Recall: 0.0958 - custom_f1: 0.1643 - auc: 0.8232 - accuracy: 0.2326 - val_loss: 38.9996 - val_Precision: 0.4103 - val_Recall: 0.0809 - val_custom_f1: 0.1343 - val_auc: 0.7064 - val_accuracy: 0.2408 - lr: 0.0020
Epoch 3/20
3158/3158 [==============================] - 233s 74ms/step - loss: 0.4190 - Precision: 0.7635 - Recall: 0.1285 - custom_f1: 0.2167 - auc: 0.8355 - accuracy: 0.2338 - val_loss: 42.6872 - val_Precision: 0.4874 - val_Recall: 0.1347 - val_custom_f1: 0.2235 - val_auc: 0.6123 - val_accuracy: 0.1179 - lr: 0.0020
Epoch 4/20
3158/3158 [======================

In [51]:
def build_model(hp):
    lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
    l1 = 0
    l2 = hp.Choice('l2',  values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
    num_hidden = hp.Int('num_hidden', 32,1024,32)
    regularizer = tf.keras.regularizers.l1_l2(l1, l2)
    model = Sequential()
    model.add(densenet_updated)
    model.add(Flatten())
    model.add(Dense(num_hidden, kernel_regularizer=regularizer))
    model.add(BatchNormalization(scale=False,
                                    center=False))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.4))
    model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss='binary_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])
    return model


In [52]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective('val_custom_f1', 'max'),
    max_trials = 10,
    overwrite=True
    
)

In [54]:
tuner.search(
    X_train, y_train, validation_data=(X_val, y_val),
    epochs=10,
    batch_size = 4
    #callbacks = scheduler
    
)

Trial 10 Complete [00h 12m 16s]
val_custom_f1: 0.29761549830436707

Best val_custom_f1 So Far: 0.344087153673172
Total elapsed time: 02h 12m 09s
INFO:tensorflow:Oracle triggered exit


In [55]:
topN = 2
for x in range(topN):
    print(tuner.get_best_hyperparameters(topN)[x].values)
    print(tuner.get_best_models(topN)[x].summary())

{'lrate': 0.02114074044975701, 'l2': 0.0, 'num_hidden': 576}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (4, 3, 3, 1024)           7031232   
                                                                 
 flatten (Flatten)           (4, 9216)                 0         
                                                                 
 dense (Dense)               (4, 576)                  5308992   
                                                                 
 batch_normalization (BatchN  (4, 576)                 1152      
 ormalization)                                                   
                                                                 
 activation (Activation)     (4, 576)                  0         
                                                                 
 dropout (Dropout)           (4, 576)                  0     

Training

In [19]:

model = Sequential()
model.add(densenet_updated)
model.add(Flatten())
model.add(Dense(576))
model.add(BatchNormalization(scale=False,
                                center=False))
model.add(Activation('relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=num_classes, activation="softmax")) 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.02114074044975701),
                loss='binary_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])


In [20]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Tuned_FACS_EmotioNet',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./clean_logs/FACS/Tuned_Model_EmotioNet")
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [21]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
    epochs=15, callbacks=callbacks)

2023-04-18 17:52:07.934502: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 465555456 exceeds 10% of free system memory.
2023-04-18 17:52:08.595170: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 465555456 exceeds 10% of free system memory.


Epoch 1/15


2023-04-18 17:52:17.509857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-18 17:52:19.147412: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-18 17:52:22.348366: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xdbf56ad0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-18 17:52:22.348412: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 970M, Compute Capability 5.2
2023-04-18 17:52:22.414236: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-18 17:52:22.792467: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-18 17:52:22.890903: I ./tensorflow/compiler/jit/device_compiler

395/395 [==============================] - ETA: 0s - loss: 0.4323 - Precision: 0.7115 - Recall: 0.1753 - custom_f1: 0.2810 - auc: 0.7926 - accuracy: 0.2085

INFO:tensorflow:Assets written to: ./chkpts/Tuned_FACS_EmotioNet/assets


INFO:tensorflow:Assets written to: ./chkpts/Tuned_FACS_EmotioNet/assets


395/395 [==============================] - 82s 172ms/step - loss: 0.4323 - Precision: 0.7115 - Recall: 0.1753 - custom_f1: 0.2810 - auc: 0.7926 - accuracy: 0.2085 - val_loss: 0.3902 - val_Precision: 0.7341 - val_Recall: 0.1781 - val_custom_f1: 0.2860 - val_auc: 0.8098 - val_accuracy: 0.1648
Epoch 2/15
395/395 [==============================] - ETA: 0s - loss: 0.3968 - Precision: 0.7476 - Recall: 0.1865 - custom_f1: 0.2982 - auc: 0.7986 - accuracy: 0.2153

INFO:tensorflow:Assets written to: ./chkpts/Tuned_FACS_EmotioNet/assets


INFO:tensorflow:Assets written to: ./chkpts/Tuned_FACS_EmotioNet/assets


395/395 [==============================] - 60s 152ms/step - loss: 0.3968 - Precision: 0.7476 - Recall: 0.1865 - custom_f1: 0.2982 - auc: 0.7986 - accuracy: 0.2153 - val_loss: 0.3922 - val_Precision: 0.7973 - val_Recall: 0.2396 - val_custom_f1: 0.3680 - val_auc: 0.7813 - val_accuracy: 0.3302
Epoch 3/15
395/395 [==============================] - 21s 54ms/step - loss: 0.3796 - Precision: 0.7552 - Recall: 0.1840 - custom_f1: 0.2961 - auc: 0.8092 - accuracy: 0.2154 - val_loss: 0.4583 - val_Precision: 0.7221 - val_Recall: 0.2251 - val_custom_f1: 0.3413 - val_auc: 0.7553 - val_accuracy: 0.3403
Epoch 4/15
395/395 [==============================] - 21s 53ms/step - loss: 0.3801 - Precision: 0.7725 - Recall: 0.1956 - custom_f1: 0.3122 - auc: 0.8017 - accuracy: 0.2200 - val_loss: 0.3979 - val_Precision: 0.6968 - val_Recall: 0.1675 - val_custom_f1: 0.2717 - val_auc: 0.8175 - val_accuracy: 0.2484
Epoch 5/15
395/395 [==============================] - 21s 54ms/step - loss: 0.3652 - Precision: 0.7923 -

In [22]:
import tensorflow as tf
model.save("./Results/FACS_tuning", save_format='h5')

In [23]:
y_pred = model.predict(X_test)

50/50 [==============================] - 7s 98ms/step


In [25]:
df = pd.DataFrame(y_pred)
df.to_csv("./Results/FACS_prediction_DenseNet")

In [26]:
df_test = pd.DataFrame(y_test)

In [28]:
df_test.to_csv("./Results/FACS_test_values")

In [43]:
df_final_predictions = pd.DataFrame({"Predicted":[df],"Real": [y_test]})

In [29]:
y_pred_clean = np.where(y_pred >= 0.5, 1, 0)

In [31]:
y_pred_clean.to_csv("./Result/FACS_prediction_clean")

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [53]:
X_test.tofile("Test_clean_EmotioNet_facs.csv")

: 

In [38]:
df.to_pickle("./FACS_Tuning_predictions")

In [ ]:
y_pred_clean = onehot_encoder.inverse_transform(y_pred)

In [ ]:
y_pred_clean = np.save("EmotioNet_Emotion_Predictions.npy")

705     5.0
1070    5.0
562     5.0
730     5.0
218     5.0
       ... 
291     5.0
918     5.0
92      4.0
885     5.0
352     2.0
Name: emotion, Length: 110, dtype: float64

In [ ]:
y_pred_clean = y_pred_clean.reshape(110)

In [ ]:
testing_results = pd.DataFrame({"Predicted value":y_pred_clean, "Real value": y_test})

In [ ]:
testing_results.to_pickle("./Final_Emotion_Results")

In [50]:
from sklearn.metrics import multilabel_confusion_matrix

result = multilabel_confusion_matrix(y_test,y_pred_clean)

In [51]:
result

array([[[1492,    0],
        [  95,    0]],

       [[1545,    0],
        [  42,    0]],

       [[1410,    7],
        [ 161,    9]],

       [[1536,    4],
        [  45,    2]],

       [[1212,    0],
        [ 375,    0]],

       [[1561,    1],
        [  25,    0]],

       [[ 475,   44],
        [ 831,  237]],

       [[ 933,    6],
        [ 549,   99]],

       [[1308,    5],
        [ 264,   10]],

       [[1556,    0],
        [  30,    1]],

       [[1322,   20],
        [ 221,   24]],

       [[1581,    0],
        [   6,    0]],

       [[ 737,  162],
        [ 273,  415]],

       [[ 739,   33],
        [ 556,  259]],

       [[1450,    0],
        [ 137,    0]],

       [[1507,    0],
        [  80,    0]]])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

pres